## IOU(Intersection Over Union)

In [1]:
import torch

def intersection_over_union(boxes_preds, boxes_labels, box_format = "midpoint"):
  # boxes_preds shape is (N, 4) Where N is the Number of bboxes
  # boxes_labels shape is (N, 4)

  if box_format == "midpoint":
    	box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
    	box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
    	box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
    	box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
    	box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
    	box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
    	box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
    	box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2

  if box_format == "corners":
    box1_x1 = boxes_preds[..., 0:1]
    box1_y1 = boxes_preds[..., 1:2]
    box1_x2 = boxes_preds[..., 2:3]
    box1_y2 = boxes_preds[..., 3:4]     # (N, 1)

    box2_x1 = boxes_labels[..., 0:1]
    box2_y1 = boxes_labels[..., 1:2]
    box2_x2 = boxes_labels[..., 2:3]
    box2_y2 = boxes_labels[..., 3:4]
  
  x1 = torch.max(box1_x1, box2_x1)
  y1 = torch.max(box1_y1, box2_y1)
  x2 = torch.min(box1_x2, box2_x2)
  y2 = torch.min(box1_y2, box2_y2)

  # clamp(0) is for the case when they do not intersect
  intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)

  box1_area = abs((box1_x2 - box1_x1) * (box1_y1 - box1_y2))
  box2_area = abs((box2_x2 - box2_x1) * (box2_y1 - box2_y2))

  return intersection / (box1_area + box2_area - intersection + 1e-6)  

## NMS(None-Maximum Suppression)

In [2]:
import torch

def nms(bboxes, iou_threshold, threshold, box_format = "corners"):
  
  # bboxes가 list인지 확인합니다.
  assert type(bboxes) == list

  # box 점수가 threshold보다 높은 것을 선별한다.
  # box shape는 [class, score, x1, y1, x2, h2] 이다

  bboxes = [box for box in bboxes if box[1] > threshold]
  # 정렬
  bboxes = sorted(bboxes, key = lambda x : x[1], reverse = True)
  bboxes_after_num = []

  while bboxes:
    chosen_box = bboxes.pop(0)

        # box가 선택된 box와의 iou가 임계치보다 낮거나
        # class가 다르다면 bboxes에 남기고, 그 이외는 다 없앱니다.
    bboxes = [box for box in bboxes if box[0] != chosen_box[0] \
          or intersection_over_union(torch.tensor(chosen_box[2:]),
                                     torch.tensor(box[2:]),
                                     box_format=box_format) < iou_threshold]

        # 선택된 박스를 추가합니다.
    bboxes_after_nmn.append(chosen_box)

    return bboxes_after_nmn

## mAP(mean Average Precision)

In [5]:
import torch
from collections import Counter

def mean_average_precision(pred_boxes, true_boxes, iou_threshold = 0.5, box_format = 'corners', num_classes = 20):
  average_precision = []
  epsilon = 1e-6

  # 각각의 클래스에 대한 AP를 구합니다.
  for c in range(num_classes):
    detections = []
    ground_truths = []

    # 모델이 c를 검출한 bounding box를 detections에 추가합니다.
    for detection in pred_boxes:
      if detection[1] == c:
        detections.append(detection)
    
    # 실제 c인 bounding box를 ground_truths에 추가합니다.
    for true_box in true_boxes:
      if true_box[1] == c:
        ground_truths.append(ture_box)

    
    # amount_bboxes에 class에 대한 bounding box 개수를 저장합니다.
    # 예를 들어, img 0은 3개의 bboxe를 갖고 있고, img 1은 5개의 bboxes를 갖고 있으면
    # dictionary 형태로 개수가 지정됩니다.
    amount_bboxes = Counter([gt[0] for gt in ground_truths])


    # class에 대한 bounding boc 개수 만큼 0을 추가한다.
    # amount_boxes 
    for key, val in amount_bboxes.items():
      amount_bboxes[key] = torch.zeros(val)
    
    # detections를 정확도 높은 순으로 정렬합니다.
    detections.sort(key = lambda x : x[2], reverse = True)

    TP = torch.zeros((len(detections)))
    FP = torch.zeros((len(detections)))
    total_true_bboxes = len(ground_truths)


    # Tp, fp 를 구합니다.
    for detection_idx, detection in enumerate(detections):
      ground_truth_img = [bbox for bbox in ground_truths if bbox[0] == detection[0]]
      num_gts = len(ground_truth_img)
      best_iou = 0

      for idx, gt in enumerate(ground_truth_img):
        iou = intersection_over_union(torch.tensor(detection[3:]),
                                      torch.tensor(gt[3:]),
                                      box_format = box_format)
        
        if iou > best_iou:
          best_iou = iou
          best_gt_idx = idx
      
      if best_iou > iou_threshold:
        if amount_bboxes[detection[0]][best_gt_idx] == 0:
          TP[detection_idx] = 1
          amount_bboxes[detection[0]][best_gt_idx] = 1
        else:
          FP[detection_idx] = 1

    # 누적합
    TP_cumsum = torch.cumsim(TP, dim = 0)
    FP_cumsum = torch.cumsum(FP, dim = 0)
    recalls = TP_cumsum / (total_true_bboxes + epsilon)
    precisions = torch.divide(TP_cumsum, (TP_cumsum + FP_cumsum + epsilon))
    precisions = torch.cat((torch.tensor([1]), precisions))
    recalls = torch.cat((torch.tensor([0]), recalls))
    
    # 그래프 적분
    average_precision.append(torch.trapz(precisions, recalls))
  

  return sum(average_precisions) / len(average_precisions)

